In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize, RandomRotation, Resize
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
train_path = '/content/drive/MyDrive/chest_xray/train'
val_path = '/content/drive/MyDrive/chest_xray/val'
test_path =  '/content/drive/MyDrive/chest_xray/test'

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
LR = 1e-3
IMG_SIZE = 224
BATCH_SIZE = 4
EPOCHS = 20

In [ ]:
train_transform = Compose([
                             
                             Resize(size=(IMG_SIZE,IMG_SIZE)), # Resizing the image to be 224 by 224
                             RandomRotation(degrees=(-20,+20)), #Randomly Rotate Images by +/- 20 degrees, Image argumentation for each epoch
                             ToTensor(), #converting the dimension from (height,weight,channel) to (channel,height,weight) convention of PyTorch
                             Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]) # Normalize by 3 means 3 StD's of the image net, 3 channels

])

validate_transform = Compose([
                             
                             Resize(size=(IMG_SIZE,IMG_SIZE)), # Resizing the image to be 224 by 224
                             ToTensor(), #converting the dimension from (height,weight,channel) to (channel,height,weight) convention of PyTorch
                             Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]) # Normalize by 3 means 3 StD's of the image net, 3 channels

])

test_transform = Compose([
                             
                             Resize(size=(IMG_SIZE,IMG_SIZE)), # Resizing the image to be 224 by 224
                             ToTensor(), #converting the dimension from (height,weight,channel) to (channel,height,weight) convention of PyTorch
                             Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]) # Normalize by 3 means 3 StD's of the image net, 3 channels

])

In [ ]:
train_dataset = datasets.ImageFolder(train_path,transform=train_transform)
print("Trainset Size:  {}".format(len(train_dataset)))

val_dataset = datasets.ImageFolder(val_path,transform=validate_transform)
print("validateset Size:  {}".format(len(val_dataset)))

#test_dataset = datasets.ImageFolder(test_path,transform=test_transform)
#print("validateset Size:  {}".format(len(test_dataset)))

Trainset Size:  5193
validateset Size:  16


In [ ]:
trainloader = DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True)
print("No. of batches in trainloader:{}".format(len(trainloader))) #Trainset Size:  5216 / batch_size: 16 = 326(No. of batches in trainloader) 
print("No. of Total examples:{}".format(len(trainloader.dataset)))

validationloader = DataLoader(val_dataset,batch_size=BATCH_SIZE,shuffle=True)
print("No. of batches in validationloader:{}".format(len(validationloader))) #validationset Size:  16 / batch_size: 16 = 1(No. of batches in validationloader) 
print("No. of Total examples:{}".format(len(validationloader.dataset)))

#testloader = DataLoader(test_dataset,batch_size=BATCH_SIZE,shuffle=True)
#print("No. of batches in testloader:{}".format(len(testloader))) #testset Size:  624 / batch_size: 16 = 39(No. of batches in testloader) 
#print("No. of Total examples:{}".format(len(testloader.dataset)))

No. of batches in trainloader:1299
No. of Total examples:5193
No. of batches in validationloader:4
No. of Total examples:16


In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size), 
            nn.ReLU(inplace=True), 
            nn.BatchNorm2d(out_channels),
            nn.Conv2d(out_channels, out_channels, kernel_size), 
            nn.ReLU(inplace=True), 
            nn.BatchNorm2d(out_channels),
        )
    def forward(self, x):
        return self.conv(x)

In [ ]:
def crop_and_concat(tens1, tens2):
    tens1_size = tens1.size()[2]
    tens2_size = tens2.size()[2]
    diff = (tens1_size - tens2_size) // 2
    tens1 = tens1[:, :, diff:tens1_size-diff, diff:tens1_size-diff]
    return torch.cat([tens1, tens2], dim=1)

In [ ]:
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()
        self.max_pool = nn.MaxPool2d(2, 2)
        # encoder
        self.down_1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=174), 
            nn.ReLU(inplace=True), 
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=3), 
            nn.ReLU(inplace=True), 
            nn.BatchNorm2d(64),
        )
        self.down_2 = DoubleConv(64, 128)
        self.down_3 = DoubleConv(128, 256)
        self.down_4 = DoubleConv(256, 512)
        # bottle_neck
        self.conv_1 = nn.Conv2d(512, 1024, 3)
        self.conv_2 = nn.Conv2d(1024, 1024, 3)
        # decoder
        self.up_conv_1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.up_1 = DoubleConv(1024, 512)
        self.up_conv_2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.up_2 = DoubleConv(512, 256)
        self.up_conv_3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.up_3 = DoubleConv(256, 128)
        self.up_conv_4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.up_4 = DoubleConv(128, 64)
        
        #output
        self.one_by_one_conv = nn.Conv2d(64,1, 1)

    def forward(self, x):
        #decoder
        x1 = self.down_1(x)
        x2 = self.max_pool(x1)

        x3 = self.down_2(x2)
        x4 = self.max_pool(x3)

        x5 = self.down_3(x4)
        x6 = self.max_pool(x5)

        x7 = self.down_4(x6)
        x8 = self.max_pool(x7)

        #bottle_neck
        x9 = self.conv_1(x8)
        x10 = self.conv_2(x9)

        #encoder
        x11 = self.up_conv_1(x10)
        x12 = crop_and_concat(x7, x11)
        x13 = self.up_1(x12)

        x14 = self.up_conv_2(x13)
        x15 = crop_and_concat(x5, x14)
        x16 = self.up_2(x15)

        x17 = self.up_conv_3(x16)
        x18 = crop_and_concat(x3, x17)
        x19 = self.up_3(x18)

        x20 = self.up_conv_4(x19)
        x21 = crop_and_concat(x1, x20)
        x22 = self.up_4(x21)
        
        #output
        x22 = self.one_by_one_conv(x22)
        return x22

In [ ]:
model = UNet().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LR)
bce = nn.BCEWithLogitsLoss()

In [ ]:
def validate(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / (
                (preds + y).sum() + 1e-8
            )
    print(f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}")
    print(f"Dice score: {dice_score/len(loader)}")

In [ ]:
def train(model, data, epochs=EPOCHS, device=DEVICE):
    for e in range(epochs):
        running_loss = 0
        for idx, (images, labels) in enumerate(data):
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            optimizer.zero_grad()
            labels = labels.unsqueeze(1)
            loss = bce(output, labels)
            loss.backward()
            optimizer.step()
        validate(model, validationloader)

In [ ]:
'''
x = torch.randn((1, 3, 224, 224)).to(DEVICE)
output = model(x)
print(output.shape)
'''
#train(model, trainloader, epochs=30)

ValueError: ignored

In [6]:
!kaggle competitions download -c carvana-image-masking-challenge

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
